# Libraries

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Importing Helper Functions
from helper_functions import drop_extraneous_col


# Recursive Feature Elimination with Cross-Validation
from sklearn.feature_selection import RFECV
# Time Series Split and GridSearchCV, where GridSearchCV is for hyperparameter tuning
# Blocked Time Series Split (which we would have to implement on our own)
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_validate
# Pipeline
from sklearn.pipeline import Pipeline
# Standard Scalar
from sklearn.preprocessing import StandardScaler
# Confusion Matrix
from sklearn.metrics import confusion_matrix

# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# XGBoost
from xgboost import XGBClassifier

# Ridge Classifier
from sklearn.linear_model import RidgeClassifier

# Support Vector Machine (SVM)
from sklearn.svm import SVC

# Using the Previous Game DataFrame

In [106]:
training_df  = pd.read_csv('csvs/cumulative_averages.csv')
drop_extraneous_col(training_df)
training_df

,team0,team1,winner,season,date,team0_encoded,team1_encoded,mp_cumulative_team0,mp_cumulative_team1,fg_cumulative_team0,...,stl%_cumulative_team1,blk%_cumulative_team0,blk%_cumulative_team1,tov%_cumulative_team0,tov%_cumulative_team1,ortg_cumulative_team0,ortg_cumulative_team1,drtg_cumulative_team0,drtg_cumulative_team1,team1_winner
0,SAC,DAL,SAC,2018,2017-10-20,23,27,240.000000,240.000000,42.000000,...,5.100000,10.500000,5.300000,15.500000,13.600000,103.900000,112.600000,109.100000,118.700000,0
1,POR,IND,POR,2018,2017-10-20,17,7,240.000000,240.000000,44.000000,...,10.600000,11.300000,14.100000,13.500000,10.800000,125.600000,123.600000,77.000000,115.700000,0
2,ORL,BRK,BRK,2018,2017-10-20,15,4,240.000000,240.000000,43.000000,...,6.200000,12.700000,2.900000,12.000000,15.600000,110.300000,115.700000,103.600000,123.600000,1
3,BOS,PHI,BOS,2018,2017-10-20,2,5,240.000000,240.000000,37.500000,...,5.800000,5.100000,8.000000,9.950000,14.400000,100.950000,110.300000,106.550000,115.100000,0
4,DET,WAS,WAS,2018,2017-10-20,8,14,240.000000,240.000000,41.000000,...,7.700000,7.000000,17.200000,7.300000,7.300000,103.600000,115.100000,91.400000,110.300000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,PHI,PHO,PHO,2024,2024-03-20,5,24,240.735294,241.470588,41.455882,...,7.380882,10.872059,10.926471,10.083824,12.583824,117.983824,118.973529,115.505882,116.622059,1
8351,UTA,OKC,OKC,2024,2024-03-20,18,16,241.838235,241.515152,42.588235,...,8.290909,10.625000,12.915152,12.877941,10.440909,117.452941,121.086364,120.983824,113.606061,1
8352,IND,DET,IND,2024,2024-03-20,7,8,240.367647,241.102941,46.382353,...,6.185294,9.545588,8.539706,10.958824,13.114706,120.725000,111.780882,119.098529,120.054412,0
8353,MIL,BOS,BOS,2024,2024-03-20,10,2,241.119403,241.838235,43.268657,...,6.738235,8.637313,11.938235,10.864179,10.339706,119.949254,124.129412,116.650746,112.086765,1


# Splitting Dataframe into Train and Test

In [107]:
undesired_columns = ['team0', 'team1', 'winner', 'season', 'date', 'team1_winner']
# We decided to train from the 2018 season to the 2023 season
training_seasons = [2018,2019,2020,2021,2022,2023]
# Splitting the dataframe into train and test
X_train = training_df[training_df['season'].isin(training_seasons)].drop(undesired_columns, axis=1)
X_test = training_df[training_df['season'] == 2024].drop(undesired_columns, axis=1)
y_train = training_df[training_df['season'].isin(training_seasons)]['team1_winner']
y_test = training_df[training_df['season'] == 2024]['team1_winner']


In [108]:
# Double checking the shapes of the training and testing dataframes

print(f'Observations in the training set: {X_train.shape[0]}')
print(f'Observations in the training set: {y_train.shape[0]}')

print(f'Observations in the testing set: {X_test.shape[0]}')
print(f'Observations in the testing set: {y_test.shape[0]}')


Observations in the training set: 7348
Observations in the training set: 7348
Observations in the testing set: 1007
Observations in the testing set: 1007


# Scaling Features

In [109]:
std_scalar = StandardScaler()
X_train = std_scalar.fit_transform(X_train)
X_test = std_scalar.fit_transform(X_test) 

# Training Models using the Previous Game DataFrame

In [110]:
model_lr = LogisticRegression()
tscv = TimeSeriesSplit()
model_rfecv = RFECV(estimator=model_lr, cv=tscv, min_features_to_select=25)
cv_results = cross_validate(model_rfecv, X_train, y_train, cv=tscv, scoring='precision', return_train_score=True)

/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:

KeyboardInterrupt: 

In [95]:
cv_results

{'fit_time': array([1.37064314, 1.72504783, 2.00561571, 2.95733523, 3.22076511]),
 'score_time': array([0.00334787, 0.0009861 , 0.00087237, 0.00087976, 0.00085974]),
 'test_score': array([0.63023256, 0.57489879, 0.55897436, 0.58002039, 0.59848485]),
 'train_score': array([0.64594895, 0.62531646, 0.61675824, 0.60610807, 0.60256141])}

# Feature Selection

In [ ]:
# This can be used for all the machine learning algorithms below
tscv = TimeSeriesSplit()
model_rfecv = RFECV(estimator=, cv=tscv, min_features_to_select=)

# Logistic Regression

In [111]:
# Define the type of cross validation
tscv = TimeSeriesSplit()
# Define the Logistic Regresion Model
model_lr = LogisticRegression(solver='saga', max_iter=1000, random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv = RFECV(estimator = model_lr, cv=tscv, min_features_to_select=30)
# Fitting the rfecv model to the data
model_rfecv.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv.transform(X_train)

/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_

In [112]:
X_train_selected.shape

(7348, 44)

In [113]:
# Define the Logistic Regresion Model
model_lr = LogisticRegression(solver='saga', max_iter=1000, random_state=42)
# Define a grid of hyperparameters
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
grid_search = GridSearchCV(estimator=model_lr, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy')
grid_search.fit(X_train_selected, y_train)

print("Best score:", grid_search.best_score_)

/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_

Best score: 0.6331699346405228


/Users/josephlim/Documents/3B/MSCI 446/MSCI-446-Project/env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# Random Forest Classifier

In [ ]:
model_rfc = RandomForestClassifier(n_estimators=, max_depth=)

# Understand features using feature_importances_


scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)

# XGBoost Classifier

In [ ]:
model_xgb = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, objective='binary:logistic')

# Understand features using feature_importances_


scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)

# Ridge Classifier

In [ ]:
# Feature selection is embedded in the algorithm
model_rc = RidgeClassifier()


scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)

# Support Vector Machine

In [ ]:
model_svm = SVC(kernel=)



scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)